In [1]:
import os
import json
import pandas as pd 
import traceback
import PyPDF2

In [2]:

from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback


In [3]:
from dotenv import load_dotenv

load_dotenv()


True

In [4]:
KEY="sk-PeKZ8vkFw2fIlQOMGPn09x_TutsBklh7H7mZsNrGF0T3BlbkFJ2s-aosngebAXKIL0c92SEzD2ocDoSs5vp85U3N6HMA"

# os.getenv("OPENAI_API_KEY")
print(KEY)

sk-PeKZ8vkFw2fIlQOMGPn09x_TutsBklh7H7mZsNrGF0T3BlbkFJ2s-aosngebAXKIL0c92SEzD2ocDoSs5vp85U3N6HMA


In [5]:
llm = ChatOpenAI(openai_api_key=KEY, model_name="gpt-4o-mini",temperature=0.5)


/Users/tusharkhari/Desktop/development/python/mcqGen/env/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [6]:
RESPONSE_JSON={
    "1":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer"
    }, 
    "2":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer"
    }, 
    "3":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer"
    },
    "4":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        }
    }, 
    "5":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer"
    }
}

In [7]:
TEMPLATE = """ 
Text:{text}
You are an expert MCQ marker. Given the above text, it is your job to /
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. / 
Ensure to make {number} MCQd
### RESPONSE_JSON 
{response_json}
"""

In [8]:
quiz_generation_prompt = PromptTemplate(input_variables=["text", "number","subject", "tone", "response_json"], template=TEMPLATE)

In [9]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

/Users/tusharkhari/Desktop/development/python/mcqGen/env/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [10]:
TEMPLATE2=""" 
you are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
    You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity 
    if the quiz is not at per with the cognitive and analytical abilities of the students, \
        update the quiz questions which needs to be changed and change the tone such that it perfectly fits students Quiz_MCQs:
        {quiz}
        
        Check from an expert English Writer of the above quiz:
"""

In [11]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)

In [12]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [13]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number","subject", "tone", "response_json" ], 
                                        output_variables=["quiz", "review"], verbose=True)

In [14]:
file_path = "/Users/tusharkhari/Desktop/development/python/mcqGen/data.txt"

In [15]:
with open(file_path, "r") as file:
    TEXT = file.read()

In [16]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "4": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}}, "5": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [17]:
NUMBER=5
SUBJECT="machine learning"
TONE="simple"

In [18]:
with get_openai_callback() as cb:
    response = generate_evaluate_chain(
        {
            "text":TEXT, 
            "number":NUMBER, 
            "subject":SUBJECT, 
            "tone":TONE, 
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )

/Users/tusharkhari/Desktop/development/python/mcqGen/env/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
 
Text:Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2]

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[3][4] When applied to business problems, it is known under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an important source of the field's methods.

The mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field of

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
quiz=response.get("quiz")


NameError: name 'response' is not defined

In [ ]:
json.loads(quiz)

NameError: name 'quiz' is not defined

In [ ]:
quiz ={
  "1": {
    "mcq": "Who coined the term machine learning?",
    "options": {
      "a": "Donald Hebb",
      "b": "Arthur Samuel",
      "c": "Walter Pitts",
      "d": "Warren McCulloch"
    },
    "correct": "b"
  },
  "2": {
    "mcq": "What was the earliest machine learning model introduced by Arthur Samuel?",
    "options": {
      "a": "Speech recognition",
      "b": "Image classification",
      "c": "Checkers game",
      "d": "Pattern recognition"
    },
    "correct": "c"
  },
  "3": {
    "mcq": "Which book introduced the Hebbian theory?",
    "options": {
      "a": "The Organization of Behavior",
      "b": "Learning Machines",
      "c": "Computing Machinery and Intelligence",
      "d": "The History of Machine Learning"
    },
    "correct": "a"
  },
  "4": {
    "mcq": "In the 1960s, a learning machine called Cybertron was developed to analyze which of the following?",
    "options": {
      "a": "Sonar signals",
      "b": "Speech patterns",
      "c": "Electrocardiograms",
      "d": "All of the above"
    },
    "correct": "d"
  },
  "5": {
    "mcq": "According to Tom M. Mitchell, what is the definition of machine learning?",
    "options": {
      "a": "Improving computer performance",
      "b": "Learning from experience to improve task performance",
      "c": "Analyzing cognitive processes",
      "d": "Developing neural networks"
    },
    "correct": "b"
  }
}




In [ ]:
quiz_json = json.dumps(quiz)

# Print the JSON string
# json.loads(quiz)
loaded_quiz = json.loads(quiz_json)
 

{'1': {'mcq': 'Who coined the term machine learning?', 'options': {'a': 'Donald Hebb', 'b': 'Arthur Samuel', 'c': 'Walter Pitts', 'd': 'Warren McCulloch'}, 'correct': 'b'}, '2': {'mcq': 'What was the earliest machine learning model introduced by Arthur Samuel?', 'options': {'a': 'Speech recognition', 'b': 'Image classification', 'c': 'Checkers game', 'd': 'Pattern recognition'}, 'correct': 'c'}, '3': {'mcq': 'Which book introduced the Hebbian theory?', 'options': {'a': 'The Organization of Behavior', 'b': 'Learning Machines', 'c': 'Computing Machinery and Intelligence', 'd': 'The History of Machine Learning'}, 'correct': 'a'}, '4': {'mcq': 'In the 1960s, a learning machine called Cybertron was developed to analyze which of the following?', 'options': {'a': 'Sonar signals', 'b': 'Speech patterns', 'c': 'Electrocardiograms', 'd': 'All of the above'}, 'correct': 'd'}, '5': {'mcq': 'According to Tom M. Mitchell, what is the definition of machine learning?', 'options': {'a': 'Improving comp

In [ ]:
quiz_table_data=[]
for key, value in loaded_quiz.items():
   mcq = value["mcq"]
   options=" | ".join(
       [
           f"{option}:{option_value}"
           for option, option_value in value["options"].items()
       ]
   )
   correct=value["correct"]
   quiz_table_data.append({"MCQ":mcq, "Choices": options, "Correct":correct})

In [ ]:
print(quiz_table_data)

[{'MCQ': 'Who coined the term machine learning?', 'Choices': 'a:Donald Hebb | b:Arthur Samuel | c:Walter Pitts | d:Warren McCulloch', 'Correct': 'b'}, {'MCQ': 'What was the earliest machine learning model introduced by Arthur Samuel?', 'Choices': 'a:Speech recognition | b:Image classification | c:Checkers game | d:Pattern recognition', 'Correct': 'c'}, {'MCQ': 'Which book introduced the Hebbian theory?', 'Choices': 'a:The Organization of Behavior | b:Learning Machines | c:Computing Machinery and Intelligence | d:The History of Machine Learning', 'Correct': 'a'}, {'MCQ': 'In the 1960s, a learning machine called Cybertron was developed to analyze which of the following?', 'Choices': 'a:Sonar signals | b:Speech patterns | c:Electrocardiograms | d:All of the above', 'Correct': 'd'}, {'MCQ': 'According to Tom M. Mitchell, what is the definition of machine learning?', 'Choices': 'a:Improving computer performance | b:Learning from experience to improve task performance | c:Analyzing cognitive

In [ ]:
quiz.to_csv("machineLearning.csv",index=False)